# MPK - clear data

#### Import 

In [112]:
import pandas as pd
from geopy.geocoders import Nominatim

#### Read data

In [180]:
df = pd.read_csv('stops.csv')
df.head()

,street_name,stops_number,stop_name_dirty
0,1 Maja,"['3033,3032']",Dubois (Zgierz)
1,1 Maja,"['3303,3304']",MDK (Aleksandrów)
2,1 Maja,"['3262,3034']",Piłsudskiego (Zgierz)
3,1 Maja,['0523'],Pogonowskiego
4,1 Maja,['3030'],plac Kilińskiego (Zgierz)


In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1333 entries, 0 to 1332
Data columns (total 3 columns):
street_name        1333 non-null object
stops_number       1333 non-null object
stop_name_dirty    1333 non-null object
dtypes: object(3)
memory usage: 31.4+ KB


#### Add NŻ (na żadanie) to df and clear it from column

In [192]:
def add_stop_type(string):
    letter_place = string.find("NŻ")
    if letter_place != -1:
        return True
    else:
        return False
    
def clear_demand_info(string):
    clear_string = string.replace("NŻ","").strip()
    return clear_string

In [185]:
df['demand_stop'] = df['stop_name_dirty'].apply(add_stop_type)
df['stop_name_dirty'] = df['stop_name_dirty'].apply(clear_demand_info)
df.head(3)

,street_name,stops_number,stop_name_dirty,demand_stop
0,1 Maja,"['3033,3032']",Dubois (Zgierz),False
1,1 Maja,"['3303,3304']",MDK (Aleksandrów),False
2,1 Maja,"['3262,3034']",Piłsudskiego (Zgierz),False


#### Clear stop name and add city

In [186]:
def get_clear_stop(string):
    letter_place = string.find("(")
    if letter_place != -1:
        clear_stop = string[0:letter_place].replace(' ','')
        return clear_stop
    else:
        return string
    
def get_city(string):
    letter_place = string.find("(")
    if letter_place != -1:
        city = string[letter_place:].replace('(','').replace(')','')
        return city
    else:
        return "Łódź"

In [189]:
df['stop_name'] = df['stop_name_dirty'].apply(get_clear_stop)
df['city'] = df['stop_name_dirty'].apply(get_city)

In [190]:
df.head()

,street_name,stops_number,stop_name_dirty,demand_stop,stop_name,city
0,1 Maja,"['3033,3032']",Dubois (Zgierz),False,Dubois,Zgierz
1,1 Maja,"['3303,3304']",MDK (Aleksandrów),False,MDK,Aleksandrów
2,1 Maja,"['3262,3034']",Piłsudskiego (Zgierz),False,Piłsudskiego,Zgierz
3,1 Maja,['0523'],Pogonowskiego,False,Pogonowskiego,Łódź
4,1 Maja,['3030'],plac Kilińskiego (Zgierz),False,placKilińskiego,Zgierz


In [196]:
df['city'].unique()

array(['Zgierz', 'Aleksandrów', 'Łódź', 'Lutomiersk', 'Brzeziny',
       'Andrespol', 'Aleks.', 'Zgierz #', 'Konst.', 'Konstantynów',
       'Dw. Ł. Kaliska', 'ZOO', 'Fala', 'Ozorków', 'Stryków', 'Pabianice',
       'kampus UŁ', 'Zg.', 'Wi-Ma', 'kampus PŁ', 'Rzgów', 'Ericsson',
       'Rąbień', 'Anton. #', 'EC1 Centr. Nauki', 'Planetarium', 'Pab.',
       'Konstantynów #', 'LO nr 1', 'Ksawerów', 'Smolice', 'Kania Góra',
       'Rosanów', 'osiedle', 'Kazimierz', 'szkoła'], dtype=object)

#### Clear city column

#### Get city 

In [89]:
condition = df['stop_name'].isin(["Z"])
df[condition]

,street_name,stops_number,stop_name


#### Find latitude and longitude

In [5]:
gc = Nominatim(user_agent="my-application")

In [6]:
location = gc.geocode('Brzezińska Kerna, Lodz, Poland')
location

Location(Brzezińska - Kerna, Dolina Łódki, Łukaszew, Łódź-Widzew, Łódź, województwo łódzkie, Polska, (51.79670005, 19.513089758813365, 0.0))

In [7]:
location = gc.geocode('Poland, Lodz, 1 Maja Piłsudskiego', exactly_one=True)
location

Location(1 Maja/Piłsudskiego, Nowe Miasto, Zgierz, powiat zgierski, województwo łódzkie, Polska, (51.86125375, 19.416210749999998, 0.0))

In [8]:
def find_location_name(address):
    try:
        location = gc.geocode(address)
        return location
    except: 
        return "NOT FOUND"

def find_location(address):
    try:
        location = gc.geocode(address)
        return location[1]
    except:
        return "NOT FOUND"
#     return {
#             'latitude':location[1][0],
#             'longitude':location[1][1]
#             }

In [9]:
location = find_location('Brzezińska Janosika, Lodz, Poland')
location

(51.7967101, 19.52299845769231)

In [10]:
stop = "Poland, Lodz, " + df.iloc[2].street_name + df.iloc[2].stop_name
print(stop)
find_location_name(stop)

Poland, Lodz, 1 Maja Piłsudskiego


Location(1 Maja/Piłsudskiego, Nowe Miasto, Zgierz, powiat zgierski, województwo łódzkie, Polska, (51.86125375, 19.416210749999998, 0.0))

In [11]:
df['tempName'] = "Poland, Lodz, " + df['street_name'] + df['stop_name']

In [12]:
df.head(3)

,street_name,stops_number,stop_name,tempName
0,1 Maja,"['3033,3032']",Dubois,"Poland, Lodz, 1 Maja Dubois"
1,1 Maja,"['3303,3304']",MDK,"Poland, Lodz, 1 Maja MDK"
2,1 Maja,"['3262,3034']",Piłsudskiego,"Poland, Lodz, 1 Maja Piłsudskiego"


In [ ]:
df['lat/lon'] = df['tempName'] .apply(find_location)
df['loc_name'] = df['tempName'] .apply(find_location_name)

### TEMP DF

In [34]:
temp_df = df.loc[1:50]
temp_df['tempName'] = "Poland, Lodz, " + temp_df['street_name'] + temp_df['stop_name']
temp_df.head()

d:\python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,street_name,stops_number,stop_name,tempName
1,1 Maja,"['3303,3304']",MDK (Aleksandrów),"Poland, Lodz, 1 Maja MDK (Aleksandrów)"
2,1 Maja,"['3262,3034']",Piłsudskiego (Zgierz),"Poland, Lodz, 1 Maja Piłsudskiego (Zgierz)"
3,1 Maja,['0523'],Pogonowskiego,"Poland, Lodz, 1 Maja Pogonowskiego"
4,1 Maja,['3030'],plac Kilińskiego (Zgierz),"Poland, Lodz, 1 Maja plac Kilińskiego (Zgierz)"
5,1 Maja,['0522'],Żeligowskiego,"Poland, Lodz, 1 Maja Żeligowskiego"


In [35]:
temp_df['lat/lon'] = temp_df['tempName'].apply(find_location)
temp_df['loc_name'] = temp_df['tempName'].apply(find_location_name)
temp_df.to_csv("TEST.csv")

d:\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
d:\python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
